In [24]:
import pandas as pd
from bs4 import BeautifulSoup
import time

from article_collection import ArticleCollector, ArticleScraper

In [35]:
import pandas as pd
import json

# Load keys
with open('../../keys.json') as f:
    keys = json.load(f)
    NEWS_API_KEY = keys["NEWSAPI_KEY3"]
    POST_NAME = keys['POST_USERNAME']
    POST_PASS = keys["POST_PASSWORD"]

scraper = ArticleScraper(POST_NAME=POST_NAME, POST_PASS=POST_PASS)

scraper_functions = {'Fox News':scraper.scrape_fox_article,
                     'CNN':scraper.scrape_CNN_article,
                     'Al Jazeera English':scraper.scrape_aljazeera_article,
                     'Washington Post':scraper.scrape_washingtonpost_article,
                     'BBC News':scraper.scrape_bbc_article,
                     'Associated Press':scraper.scrape_ap_article,
                     }
texts = []

articles_df = pd.read_csv('data/corpus_info.csv')
articles_df = articles_df.drop_duplicates(subset=['url'])
articles_df = articles_df[articles_df.source == 'Washington Post']

articles_df = articles_df.iloc[0:3]



In [36]:
articles_df.source.value_counts()

source
Washington Post    3
Name: count, dtype: int64

In [ ]:
for i in range(len(articles_df)):
    time.sleep(0.5)
    series = articles_df.iloc[i]
    url = series.url
    source = series.source

    if source != 'Washington Post': 
        text = scraper_functions[source](url)
    else:
        text = 'SCRAPE LATER'


    texts.append(text)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.86)
Stacktrace:
0   chromedriver                        0x0000000101363ac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x000000010135c314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x0000000100dc44b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000100d9f994 core::str::slice_error_fail::ha0e52dbcb60e6bae + 3780
4   chromedriver                        0x0000000100e2e890 cxxbridge1$string$len + 524388
5   chromedriver                        0x0000000100e4176c cxxbridge1$string$len + 601920
6   chromedriver                        0x0000000100dfd0b0 cxxbridge1$string$len + 321668
7   chromedriver                        0x0000000100dfdd00 cxxbridge1$string$len + 324820
8   chromedriver                        0x000000010132ee08 cxxbridge1$str$ptr + 3435328
9   chr

In [14]:
texts[-1]

'Donald Trump has declared victory in the 2024 US presidential election, after several battleground states were called in his favour. He spoke to supporters in Florida.'